# [데이터 전처리/시각화_2회차]
결측·중복·문자열 지옥 탈출: Pandas 클리닝 핵심

## 오늘의 목표
“카페 매출 데이터에서 필요한 행/열만 정확히 뽑고, 조건으로 걸러 정렬한 뒤, 결측·중복·문자열 문제를 정리해서 ‘분석 가능한 테이블’ 1개를 만든다.”

## 왜 분석부터 하는것이 아닌 분석 가능한 테이블을 먼저 만드는가?
- 분석은 "깨끗한 테이블" 정확히는 정제된 데이터가 있어야지만 의미가 생긴다.
- 실무의 데이터들은 대부분 정제되지 않은 '원시 데이터'이기 때문에 분석에 바로 사용하기 어렵다
    - 원시데이터: 날짜 형식이 섞여있거나, 금액에 원 또는 쉽표로 분리되어 있거나
    - 값이 누락되어 있다. (결측값)
- 오늘은 이러한 원시데이터를 조회 → 필터 → 정렬 → 클리닝 순으로 해결하는 방법을 알아보려 한다.


## 실습 준비: 카페 매출 샘플 데이터 만들기 (일부러 문제를 섞어둠)

### 시나리오
당신은 카페 사장님/매니저입니다.

**“이번 주 판매 TOP 메뉴와 시간대 매출”** 을 내일 아침 회의에서 보고해야 합니다.

허나 원본 데이터가 정제되지 않아 분석하기엔 문제가 많습니다.

In [1]:
# 데이터 프레임 생성
import pandas as pd

raw = [
    {"date":"2026-01-01", "time":"09:10", "store":"A", "menu":"Americano", "price":"4,500원", "qty":"2", "paid":"TRUE"},
    {"date":"2026/01/01", "time":"09:12", "store":"A", "menu":"Latte",     "price":"5000",   "qty":1,   "paid":"True"}, # /
    {"date":"2026-01-02", "time":"12:30", "store":"A", "menu":"Latte",     "price":None,     "qty":2,   "paid":"FALSE"},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},  # 중복
    {"date":"2026-01-04", "time":"08:55", "store":"B", "menu":"Americano ", "price":"4500",  "qty":"1", "paid":"TRUE"}, # 공백
    {"date":"2026-01-04", "time":"08:58", "store":"A", "menu":"latte",     "price":"5,000",  "qty":"3", "paid":"TRUE"}, # 소문자
]
df = pd.DataFrame(raw)

df

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE
3,2026-01-03,18:05,B,Mocha,5500,None,True
4,2026-01-03,18:05,B,Mocha,5500,None,True
5,2026-01-04,08:55,B,Americano,4500,1,TRUE
6,2026-01-04,08:58,A,latte,"5,000",3,TRUE


### 1회차 내용 기억하기!
```python
df.shape
df.info()
df.describe(include="all")
```

In [2]:
# 데이터프레임의 크기(행, 열)를 튜플의 형태로 알려준다.
# 데이터의 크기를 빠르게 파악 가능하게 한다.
df.shape

(7, 7)

In [3]:
# 데이터프레임의 전체 요약 정보를 출력한다.
# 행 개수, 열 개수, 각 컬럼의 이름, 데이터 타입, 결측치의 여부, 메모리 사용량 등을 확인 할수 있다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    7 non-null      object
 1   time    7 non-null      object
 2   store   7 non-null      object
 3   menu    7 non-null      object
 4   price   6 non-null      object
 5   qty     5 non-null      object
 6   paid    7 non-null      object
dtypes: object(7)
memory usage: 520.0+ bytes


#### describe의 해석 방법
describe() 결과는 “각 컬럼의 데이터 상태를 요약해서 보여주는 표”다.


1. count: “어떤 컬럼에 결측치가 있나?” 를 확인하는 용도
count = 실제 값이 들어있는 개수
    - 전체 행 수와 비교한다
    - 행 수보다 작으면 → 결측치(NaN)가 있다는 뜻


2. unique: “이 컬럼은 범주형인가?”
unique = 서로 다른 값의 개수
    - 값 종류가 얼마나 되는지 파악
    - 너무 많으면 → 식별자(ID)일 가능성
    - 너무 적으면 → 범주형 데이터일 가능성


3. top, freq: 데이터가 한쪽으로 치우쳐 있는지, 대표값이 있는 컬럼인지
- top = 가장 많이 나온 값 (최빈값)
- frep = 그 값이 몇 번 나왔는지


4. 숫자 컬럼 vs 문자 컬럼
- 문자(범주형) → count / unique / top / freq 위주
- 숫자형 → mean / std / min / max 같은 통계값
- include="all" → 숫자든 문자든 가능한 요약 전부 확인



이 표를 통해 다음을 확인할수 있다.
- 결측치 있는 컬럼은?
- 값 종류가 이상하게 많은 컬럼은?
- 범주가 통일 안 된 컬럼은?
- 대표값이 있는 컬럼은?

In [4]:
# 데이터의 통계 요약을 보여준다.
# 수치형(숫자) 컬럼: count, mean, std, min, max, 사분위수(25%, 50%, 75%)
# 범주형(문자열) 컬럼: count, unique, top(최빈값), freq
# include="all"을 쓰면 숫자형 + 범주형 컬럼을 모두 요약
df.describe(include="all")

,date,time,store,menu,price,qty,paid
count,7,7,7,7,6,5,7
unique,5,6,2,5,5,5,4
top,2026-01-04,18:05,A,Latte,5500,2,TRUE
freq,2,2,4,2,2,1,3


##### 검사 결과 해석
- 결측치 존재
    - price: 1개
    - qty: 2개
- 범주 통일 필요
    - price, qty, paid가 숫자/불리언이 아닌 문자열로 섞여 있을 가능성이 있음

- 중복 행이 존재
- 메뉴명에 공백/대소문자


### 인덱싱 핵심: .loc vs .iloc
인덱싱이란?
- 데이터프레임에서 특정 행(row)과 열(column)을 선택하는 방법

#### .loc = 라벨(이름) 기반 (행/열 이름으로 접근)
이 이름을 가진 행/열을 가져와라
- 행/열 이름을 사용
- 문자열 컬럼명 사용 가능
- 슬라이싱 시 끝 값 포함
- 조건 필터링과 함께 자주 사용됨


#### .iloc = 위치(순서) 기반 (0번째, 1번째…로 접근)
몇 번째에 있는 걸 가져와라
- 정수 인덱스만 사용
- 파이썬 리스트 인덱싱과 동일한 규칙
- 슬라이싱 시 끝 값 미포함
- 구조가 확실할 때 사용하기 좋음    

In [5]:
df

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE
3,2026-01-03,18:05,B,Mocha,5500,None,True
4,2026-01-03,18:05,B,Mocha,5500,None,True
5,2026-01-04,08:55,B,Americano,4500,1,TRUE
6,2026-01-04,08:58,A,latte,"5,000",3,TRUE


In [6]:
# 실습 1: “한 행” 뽑기
df.loc[0]      # 0번 인덱스 라벨

date     2026-01-01
time          09:10
store             A
menu      Americano
price        4,500원
qty               2
paid           TRUE
Name: 0, dtype: object

In [7]:
# 실습 1: “한 행” 뽑기
df.iloc[1]     # 1번째 위치

date     2026/01/01
time          09:12
store             A
menu          Latte
price          5000
qty               1
paid           True
Name: 1, dtype: object

In [8]:
# 실습 2: “행 범위” 슬라이싱 실수 포인트
df.loc[0:2]    # 끝 포함

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE


In [9]:
# 실습 2: “행 범위” 슬라이싱 실수 포인트
df.iloc[1:3]   # 끝 미포함

,date,time,store,menu,price,qty,paid
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE


### 열 선택 실전: 필요한 컬럼만 남기기
회의용 테이블엔 다음 컬럼만 필요로 합니다.
- date, time, store, menu, price, qty, paid

In [10]:
cols = ["date","time","store","menu","price","qty","paid"]
df2 = df[cols].copy()
df2.head()

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE
3,2026-01-03,18:05,B,Mocha,5500,None,True
4,2026-01-03,18:05,B,Mocha,5500,None,True


### 조건 필터링(불리언 인덱싱): “결제 완료 + A매장 + 오전”
규칙 1) & | ~ 는 괄호 필수

규칙 2) 실무 패턴은 이거 하나: df.loc[조건, 컬럼]


In [11]:
#Step 1) 먼저 paid 값을 통일(소문자/대문자/True 섞임)
df2["paid"] = df2["paid"].astype(str).str.strip().str.lower()
df2["paid"].value_counts()

paid
true     6
false    1
Name: count, dtype: int64

In [12]:
# Step 2) 결제 완료만 추출
cond_paid = df2["paid"].isin(["true"])
paid_df = df2.loc[cond_paid]
paid_df

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,true
1,2026/01/01,09:12,A,Latte,5000,1,true
3,2026-01-03,18:05,B,Mocha,5500,None,true
4,2026-01-03,18:05,B,Mocha,5500,None,true
5,2026-01-04,08:55,B,Americano,4500,1,true
6,2026-01-04,08:58,A,latte,"5,000",3,true


In [13]:
# Step 3) A매장 + 오전(09시대)만 더 좁히기
cond_storeA = (df2["store"] == "A")
cond_morning = df2["time"].str.startswith("09")

df_morning_A = df2.loc[cond_paid & cond_storeA & cond_morning, ["date","time","store","menu","price","qty"]]
df_morning_A

,date,time,store,menu,price,qty
0,2026-01-01,09:10,A,Americano,"4,500원",2
1,2026/01/01,09:12,A,Latte,5000,1


### 정렬: “메뉴별 판매수량 TOP”을 뽑을 준비
- sort_values = 값 기준 정렬
- sort_index = 인덱스 기준 정렬

In [14]:
df2.sort_values(by=["date","time"], ascending=[True, True]).head()

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,true
2,2026-01-02,12:30,A,Latte,None,2,false
3,2026-01-03,18:05,B,Mocha,5500,None,true
4,2026-01-03,18:05,B,Mocha,5500,None,true
5,2026-01-04,08:55,B,Americano,4500,1,true


### 클리닝 기본기 4종 세트 (오늘의 하이라이트)
- rename / drop: 컬럼명 정리(선택)
    실무에서는 컬럼명 통일을 중요시합니다.

- 문자열 정리: menu 공백/대소문자 통일

- price 정리: “원”, “,” 제거하고 숫자로 만들기


- 결측치 처리: dropna vs fillna (전략 선택)
    - 가격(price)이 비어 있으면 매출 계산이 안 되기 때문에
        - 삭제(drop): 데이터가 적어도 괜찮고, 결측이 중요 변수면 제거
        - 대체(fill): 평균/중앙값/메뉴별 평균 등으로 대체

In [15]:
# rename / drop: 컬럼명 정리(선택)
df2 = df2.rename(columns={"qty":"quantity"})

In [16]:
# 문자열 정리: menu 공백/대소문자 통일
df2["menu"] = df2["menu"].astype(str).str.strip().str.title()
df2["menu"].value_counts()

menu
Latte        3
Americano    2
Mocha        2
Name: count, dtype: int64

In [17]:
# price 정리: “원”, “,” 제거하고 숫자로 만들기
df2["price"] = (
    df2["price"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
    .str.strip()
)

df2["price"] = pd.to_numeric(df2["price"], errors="coerce")
df2["price"].isnull().sum()

np.int64(1)

#### ※ 문자열 패턴에 따라 정규식(Regular Expression)을 사용할 수 있음
```python
df2["price"] = (
    df2["price"]
    .astype(str)
    .str.replace(r"[^0-9]", "", regex=True)  # 숫자(0-9) 제외 전부 제거
)

df2["price"] = pd.to_numeric(df2["price"], errors="coerce")
df2["price"].isnull().sum()
```

In [18]:
# 결측치 처리: dropna vs fillna (전략 선택)
# # quantity도 숫자로 정리
df2["quantity"] = pd.to_numeric(df2["quantity"], errors="coerce")

In [19]:
# 결측치 처리: dropna vs fillna (전략 선택)
# 핵심 변수 결측은 삭제(간단 버전)
clean = df2.dropna(subset=["price","quantity"]).copy()
clean.shape

(4, 7)

### 중복 처리: duplicated / drop_duplicates
기대 결과
- 완전히 같은 행이 있으면 제거해서 “실제 판매 기록”에 가까워짐

In [20]:
# 중복 처리
clean.duplicated().sum()
clean = clean.drop_duplicates(keep="first")
clean.duplicated().sum()

np.int64(0)

### 9) 미니 결과물 만들기: “매출 컬럼 추가 + 회의용 테이블 정리”

In [21]:
clean["sales"] = clean["price"] * clean["quantity"]

meeting_table = clean.loc[:, ["date","time","store","menu","price","quantity","sales"]].sort_values(
    by=["date","store","time"],
    ascending=[True, True, True])

meeting_table

,date,time,store,menu,price,quantity,sales
0,2026-01-01,09:10,A,Americano,4500.0,2.0,9000.0
6,2026-01-04,08:58,A,Latte,5000.0,3.0,15000.0
5,2026-01-04,08:55,B,Americano,4500.0,1.0,4500.0
1,2026/01/01,09:12,A,Latte,5000.0,1.0,5000.0


### 10) 저장: 2회차 결과물을 파일로 남기기 (3회차를 위해 필수)

In [22]:
meeting_table.to_csv("cafe_sales_clean_v1.csv", index=False, encoding="utf-8-sig")

## 실습 풀이

cafe_sales_clean_v1.csv 파일 생성아래 조건을 만족하는 테이블 캡처(또는 출력)
    - 결제 완료(true)
    - store == "A"
    - menu는 공백/대소문자 정리 완료
    - sales 컬럼 포함
    - 한 줄 소감(필수):
        - “오늘 가장 헷갈렸던 포인트 1개 + 이유 1개”

cafe_sales_clean_v1.csv 파일 생성

아래 조건을 만족하는 테이블 캡처(또는 출력)
    - 결제 완료(true)
    - store == "A"
    - menu는 공백/대소문자 정리 완료
    - sales 컬럼 포함
    - 한 줄 소감(필수):
        - “오늘 가장 헷갈렸던 포인트 1개 + 이유 1개”

In [23]:
import pandas as pd

raw = [
    {"date":"2026-01-01", "time":"09:10", "store":"A", "menu":"Americano", "price":"4,500원", "qty":"2", "paid":"TRUE"},
    {"date":"2026/01/01", "time":"09:12", "store":"A", "menu":"Latte",     "price":"5000",   "qty":1,   "paid":"True"}, # /
    {"date":"2026-01-02", "time":"12:30", "store":"A", "menu":"Latte",     "price":None,     "qty":2,   "paid":"FALSE"},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},  # 중복
    {"date":"2026-01-04", "time":"08:55", "store":"B", "menu":"Americano ", "price":"4500",  "qty":"1", "paid":"TRUE"}, # 공백
    {"date":"2026-01-04", "time":"08:58", "store":"A", "menu":"latte",     "price":"5,000",  "qty":"3", "paid":"TRUE"}, # 소문자
]
df = pd.DataFrame(raw)

df

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE
3,2026-01-03,18:05,B,Mocha,5500,None,True
4,2026-01-03,18:05,B,Mocha,5500,None,True
5,2026-01-04,08:55,B,Americano,4500,1,TRUE
6,2026-01-04,08:58,A,latte,"5,000",3,TRUE


해야할 포인트

- 중복 행 제거
- paid를 진짜 불리언(True/False)로 통일
- menu 공백 제거 + 대소문자 통일
- price, qty를 숫자로 통일(결측치 처리 포함)
- sales = price * qty 컬럼 만들고
    - paid == True, store == "A" 조건으로 필터링
- cafe_sales_clean_v1.csv 파일 생성

In [24]:
# 중복 제거
df = df.drop_duplicates()

# menu 정리: 공백 제거 + 대소문자 통일
df["menu"] = df["menu"].astype("string").str.strip().str.lower().str.title()

# paid 정리: True/False로 통일
# - True/False(bool) + "TRUE"/"False"(str) 섞인 상태를 모두 처리
paid_str = df["paid"].astype("string").str.upper()
df["paid"] = paid_str.map({"TRUE": True, "FALSE": False})


# price 정리: "원", "," 제거 후 숫자 변환
df["price"] = (
    df["price"]
      .astype("string")
      .str.replace("원", "", regex=False)
      .str.replace(",", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

# qty 정리: 숫자 변환
df["qty"] = pd.to_numeric(df["qty"], errors="coerce")

# 결측치 처리(연습용): price/qty 결측은 0으로
df[["price", "qty"]] = df[["price", "qty"]].fillna(0)

# sales 컬럼 생성
df["sales"] = df["price"] * df["qty"]

# 조건 필터링: 결제 완료(True) + store A
result = df[(df["paid"] == True) & (df["store"] == "A")].copy()


C:\Users\joon5\AppData\Local\Temp\ipykernel_55120\2770258190.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["menu"] = df["menu"].astype("string").str.strip().str.lower().str.title()
C:\Users\joon5\AppData\Local\Temp\ipykernel_55120\2770258190.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["paid"] = paid_str.map({"TRUE": True, "FALSE": False})
C:\Users\joon5\AppData\Local\Temp\ipykernel_55120\2770258190.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [25]:
# CSV 저장
result.to_csv("cafe_sales_clean_v1.csv", index=False, encoding="utf-8-sig")

# 결과 출력(캡처용)
print(result)

         date   time store       menu  price  qty  paid    sales
0  2026-01-01  09:10     A  Americano   4500  2.0  True   9000.0
1  2026/01/01  09:12     A      Latte   5000  1.0  True   5000.0
6  2026-01-04  08:58     A      Latte   5000  3.0  True  15000.0
